In [31]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from  pandas import DataFrame
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from pandas import Series, ExcelWriter
import os
import warnings
warnings.filterwarnings("ignore")

In [32]:
cd "C:\Users\Administrator\Desktop\EEG_decoding_digit_Span\提取主成分后预测"

C:\Users\Administrator\Desktop\EEG_decoding_digit_Span\提取主成分后预测


In [33]:
#read data

data = pd.read_excel('adult_child_rest_task _WISC.xlsx', 'Sheet1')
data.shape

(151, 219)

In [34]:
open = data.columns[1:76]
open_delta = data.columns[1:16]
open_theta = data.columns[16:31]
open_alpha = data.columns[31:46]
open_beta = data.columns[46:61]
open_gamma = data.columns[61:76]
close = data.columns[76:151]
close_delta = data.columns[76:91]
close_theta = data.columns[91:106]
close_alpha = data.columns[106:121]
close_beta = data.columns[121:136]
close_gamma = data.columns[136:151]
alpha_peak = data.columns[151:166]
A_mmn = data.columns[166:176]
AV_mmn = data.columns[176:186]
AV_p3a = data.columns[186:196]
rest = data.columns[1:151]
task = data.columns[176:196]
EEG = data.columns[1:196]

In [55]:
#define our data
#feature = data.columns[1:166]
X_close_AV_mmn = np.hstack([data[close], data[AV_mmn]])
y_ds_forward = data["ds_forward_maximum_length"]
y_ds_backward = data["ds_backward_maximum_length"]
y_ds = data["WISC_digit_span"]
X = data[task]
X.head(5)

,AV_mmn_F3,AV_mmn_F1,AV_mmn_FZ,AV_mmn_F2,AV_mmn_F4,AV_mmn_FC3,AV_mmn_FC1,AV_mmn_FCZ,AV_mmn_FC2,AV_mmn_FC4,AV_p3a_F3,AV_p3a_F1,AV_p3a_FZ,AV_p3a_F2,AV_p3a_F4,AV_p3a_FC3,AV_p3a_FC1,AV_p3a_FCZ,AV_p3a_FC2,AV_p3a_FC4
0,-0.516522,-0.418504,-0.214447,0.834332,-0.126939,-1.962154,0.130807,0.425515,0.997456,0.939337,6.222157,5.876725,5.072663,6.412089,5.902603,5.552392,8.145100,7.634656,7.478193,6.358266
1,-1.074383,-0.823153,0.568924,0.758849,-0.990181,0.375565,1.435351,1.320000,0.906645,-1.326902,0.975027,3.859861,7.663907,7.271880,3.805610,2.028494,5.226977,5.419051,6.192372,3.521240
2,-2.501307,-3.352767,-3.223858,-3.049875,-4.035750,-5.929013,-6.849557,-5.601593,-4.196994,-3.267915,9.070921,9.688465,9.735043,9.357132,8.047412,0.690119,5.575251,7.160330,7.703257,8.491722
3,-4.952323,-4.516264,-3.035848,-1.336977,-1.066085,-3.588016,-2.395822,-1.492748,-1.218845,-1.378288,4.016657,5.377981,6.739858,8.333962,8.749174,5.888369,8.304365,9.282230,8.554861,6.891946
4,-0.297048,-0.090058,3.313568,0.536918,-0.529483,-0.532082,0.640125,0.483879,-0.166688,-1.073003,-1.896331,1.014123,7.153675,3.088736,2.798402,-1.982623,2.994873,5.570593,4.056171,1.128345


In [56]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components = 20, kernel="rbf")
X_reduced = rbf_pca.fit_transform(X)
X_reduced.shape

(151, 20)

In [57]:
#想要输出的值，feature importance，correlation between predict value and real value, r_2 in train and test, RMSE in train and test
fi = []
cor = []
score_train = []
score_test = []
RMSE_train = []
RMSE_test = []
MAPE = []

for i in range(100): 
    
    #fit model, tune parameter
    X_train,X_test,y_train,y_test= train_test_split(X_reduced,y_ds_forward,test_size=0.20)
    encv = ElasticNetCV(alphas=[ 0.001,0.007,0.009, 0.01,0.07, 0.09, 0.1,0.15, 0.2, 0.3, 0.5, 0.7, 1], 
                        l1_ratio=[0.05,0.07, 0.09,.1,0.15,0.2, 0.3, 0.4, .5,0.6, .7,0.8, .9, .95, .99, .995, 1], normalize=True, cv = 5)
    encv.fit(X_train, y_train)
    print('ElasticNet optimal alpha: %.3f and L1 ratio: %.4f' % (encv.alpha_, encv.l1_ratio_))
    y_train_pred = encv.predict(X_train)
    y_pred = encv.predict(X_test)
    
    #calculate correlation
    a = np.vstack((y_test, y_pred))
    b = np.corrcoef(a)
    r = b[1,0]
    print('corr_pred_test:%.4f' % r)
    
    #calculate MAPE
    n = len(y_test)
    mape = np.mean(np.abs((y_test - y_pred)/y_test))
    print('mape:%.4f' % mape)
    
    #estimate model performance in train and test
    train_score = encv.score(X_train, y_train)
    test_score= encv.score(X_test, y_test)
    print('Train score: %.4f' % train_score)
    print('Test score: %.4f' % test_score)
    train_mse = mean_squared_error(y_train_pred, y_train)
    test_mse = mean_squared_error(y_pred, y_test)
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    print('Train RMSE: %.4f' % train_rmse)
    print('Test RMSE: %.4f' % test_rmse)
    
    #if we want to output the feature importance of each circle in different sheet of the same excel, we can do like that
    #feature_importance = pd.Series(index = X_train.columns, data = np.abs(encv.coef_))
    #print(feature_importance)
    # n_selected_features = (feature_importance>0).sum()
    #print('{0:d} features, reduction of {1:2.2f}%'.format(n_selected_features,(1-n_selected_features/len(feature_importance))*100))
    #feature_importance.sort_values().tail(60).plot(kind = 'bar', figsize = (12,5))
    #sheet_name = 'Sheet%s' % i
    #Series.to_frame(feature_importance,name = '').to_excel(fi, sheet_name=sheet_name)#将每次循环的特征及其重要性存到sheet里
    
    #we can also repeat the foward step in an easy way
    #calculate feature importance score
    fi_score = np.abs(encv.coef_)
    fi.append(fi_score)
    
    cor.append(r)
    MAPE.append(mape)
    score_train.append(train_score)
    score_test.append(test_score)
    RMSE_train.append(train_rmse)
    RMSE_test.append(test_rmse)


#save data to excel
all = cor+MAPE+score_train+score_test+RMSE_train+RMSE_test
all_mat = np.mat(all)#transform list to matrix
all_reshape = all_mat.reshape(6,100).T  #reshape the matrix so that each row represent a circle and columns value match with all
df = DataFrame(all_reshape, columns = ["cor","MAPE","score_train","score_test","RMSE_train","RMSE_test"])   #transform the matrix to dataframe and add label
df.to_excel("results_task_forward.xlsx")#save the output to excel

#save the feature importance score in each circle and save to the excel
fi_mat = np.mat(fi)
fi_reshape = fi_mat.T
fi_dataframe = DataFrame(fi_reshape, index = X_train.columns)
fi_dataframe.to_excel("results_fi_task_forward.xlsx")

ElasticNet optimal alpha: 0.010 and L1 ratio: 0.8000
corr_pred_test:0.5060
mape:0.1083
Train score: 0.1827
Test score: 0.1705
Train RMSE: 1.1836
Test RMSE: 1.1303
ElasticNet optimal alpha: 0.009 and L1 ratio: 1.0000
corr_pred_test:0.1205
mape:0.1229
Train score: 0.2795
Test score: -0.0867
Train RMSE: 1.0957
Test RMSE: 1.3493
ElasticNet optimal alpha: 0.010 and L1 ratio: 0.6000
corr_pred_test:0.2983
mape:0.0996
Train score: 0.2146
Test score: 0.0742
Train RMSE: 1.2039
Test RMSE: 0.9706
ElasticNet optimal alpha: 0.010 and L1 ratio: 0.9000
corr_pred_test:0.2761
mape:0.0919
Train score: 0.2173
Test score: 0.0568
Train RMSE: 1.2079
Test RMSE: 0.9513
ElasticNet optimal alpha: 0.010 and L1 ratio: 1.0000
corr_pred_test:0.4475
mape:0.1205
Train score: 0.1491
Test score: 0.1363
Train RMSE: 1.1607
Test RMSE: 1.3225
ElasticNet optimal alpha: 0.007 and L1 ratio: 0.3000
corr_pred_test:0.3749
mape:0.1065
Train score: 0.2224
Test score: -0.1555
Train RMSE: 1.0916
Test RMSE: 1.4034
ElasticNet optimal a

corr_pred_test:0.1849
mape:0.1278
Train score: 0.2831
Test score: -0.0288
Train RMSE: 1.0541
Test RMSE: 1.4843
ElasticNet optimal alpha: 0.010 and L1 ratio: 1.0000
corr_pred_test:0.3376
mape:0.1138
Train score: 0.1764
Test score: 0.0943
Train RMSE: 1.1297
Test RMSE: 1.3880
ElasticNet optimal alpha: 0.009 and L1 ratio: 0.9000
corr_pred_test:0.2142
mape:0.0959
Train score: 0.2542
Test score: -0.0484
Train RMSE: 1.1640
Test RMSE: 1.0844
ElasticNet optimal alpha: 0.010 and L1 ratio: 1.0000
corr_pred_test:-0.0065
mape:0.1302
Train score: 0.3060
Test score: -0.1965
Train RMSE: 1.0809
Test RMSE: 1.3999
ElasticNet optimal alpha: 0.009 and L1 ratio: 0.0900
corr_pred_test:0.0295
mape:0.1358
Train score: 0.2812
Test score: -0.0422
Train RMSE: 1.0498
Test RMSE: 1.5241
ElasticNet optimal alpha: 0.007 and L1 ratio: 0.0500
corr_pred_test:0.4018
mape:0.1372
Train score: 0.2285
Test score: 0.1127
Train RMSE: 1.0852
Test RMSE: 1.4063
ElasticNet optimal alpha: 0.010 and L1 ratio: 1.0000
corr_pred_test:0.

AttributeError: 'numpy.ndarray' object has no attribute 'columns'